In [125]:
import os

import numpy as np
import xarray as xr
import iris

from ml_downscaling_emulator import UKCPDatasetMetadata

In [205]:
ywind_meta = UKCPDatasetMetadata(os.getenv("MOOSE_DATA"), domain="uk", resolution="2.2km", variable="ywind", frequency="day")
lsrain_meta = UKCPDatasetMetadata(os.getenv("MOOSE_DATA"), domain="uk", resolution="2.2km", variable="lsrain", frequency="day")
year = 1981

In [127]:
ywind_ds = xr.open_dataset(ywind_meta.filepath(year))
ywind_ds

<xarray.Dataset>
Dimensions:                     (time: 360, pressure: 17, grid_latitude: 655,
                                 grid_longitude: 532, bnds: 2)
Coordinates:
  * time                        (time) object 1980-12-01 12:00:00 ... 1981-11...
  * pressure                    (pressure) float32 10.0 20.0 ... 925.0 1e+03
  * grid_latitude               (grid_latitude) float32 -5.565 -5.529 ... 8.945
  * grid_longitude              (grid_longitude) float32 353.1 353.1 ... 365.2
    forecast_period             (time) timedelta64[ns] ...
    forecast_reference_time     object ...
    realization                 int32 ...
Dimensions without coordinates: bnds
Data variables:
    y_wind                      (time, pressure, grid_latitude, grid_longitude) float32 ...
    rotated_latitude_longitude  int32 ...
    time_bnds                   (time, bnds) object ...
    grid_latitude_bnds          (grid_latitude, bnds) float32 ...
    grid_longitude_bnds         (grid_longitude, bnds) float32 ...
    forecast_period_bnds        (time, bnds) float64 ...
Attributes:
    source:       Data from Met Office Unified Model
    um_version:   10.6
    Conventions:  CF-1.7

In [128]:
ywind_cube = iris.load_cube(ywind_meta.filepath(year))
ywind_cube

<iris 'Cube' of y_wind / (m s-1) (time: 360; pressure: 17; grid_latitude: 655; grid_longitude: 532)>

In [228]:
xr.DataArray.from_iris(ywind_cube)

,Array,Chunk
Bytes,7.94 GiB,112.99 MiB
Shape,"(360, 17, 655, 532)","(5, 17, 655, 532)"
Count,145 Tasks,72 Chunks
Type,float32,numpy.ndarray


In [129]:
def reopen_as_cube(ds):
    ds.to_netcdf("data.nc")
    return iris.load_cube("data.nc")

In [235]:
ds = ywind_ds.isel(time=slice(2))
reopen_as_cube(ds)

<iris 'Cube' of y_wind / (m s-1) (time: 2; pressure: 17; grid_latitude: 655; grid_longitude: 532)>

In [236]:
ds = ds.sel(pressure=850.0)
reopen_as_cube(ds)

<iris 'Cube' of y_wind / (m s-1) (time: 2; grid_latitude: 655; grid_longitude: 532)>

In [237]:
ds = ds.assign({'y_wind': xr.DataArray.from_iris(iris.load_cube('data.nc'))})
ds

<xarray.Dataset>
Dimensions:                     (time: 2, grid_latitude: 655,
                                 grid_longitude: 532, bnds: 2)
Coordinates:
  * time                        (time) object 1980-12-01 12:00:00 1980-12-02 ...
    pressure                    float32 850.0
  * grid_latitude               (grid_latitude) float32 -5.565 -5.529 ... 8.945
  * grid_longitude              (grid_longitude) float32 353.1 353.1 ... 365.2
    forecast_period             (time) timedelta64[ns] 450 days 12:00:00 451 ...
    forecast_reference_time     object 1979-09-01 00:00:00
    realization                 int32 1100000
Dimensions without coordinates: bnds
Data variables:
    y_wind                      (time, grid_latitude, grid_longitude) float32 dask.array<chunksize=(2, 655, 532), meta=np.ndarray>
    rotated_latitude_longitude  int32 -2147483647
    time_bnds                   (time, bnds) object 1980-12-01 00:00:00 ... 1...
    grid_latitude_bnds          (grid_latitude, bnds) float32 -5.583 ... 1.07...
    grid_longitude_bnds         (grid_longitude, bnds) float32 353.1 ... 365.2
    forecast_period_bnds        (time, bnds) float64 1.08e+04 ... 1.085e+04
Attributes:
    source:       Data from Met Office Unified Model
    um_version:   10.6
    Conventions:  CF-1.7

In [140]:
ds = ds.rename({'y_wind': 'ywind'})
reopen_as_cube(ds)

<iris 'Cube' of y_wind / (m s-1) (time: 2; grid_latitude: 655; grid_longitude: 532)>

In [141]:
from ml_downscaling_emulator.preprocessing.coarsen import Coarsen

In [142]:
ds = Coarsen(scale_factor=8).run(ds)
reopen_as_cube(ds)

<iris 'Cube' of y_wind / (m s-1) (time: 2; grid_latitude: 81; grid_longitude: 66)>

In [244]:
ds = ywind_ds.isel(time=slice(2)).sel(pressure=850.0).rename({'y_wind': 'ywind'})
ds = Coarsen(scale_factor=8).run(ds)
reopen_as_cube(ds)
cube = iris.load_cube('data.nc')
ywind_regrid_cube = iris.analysis.Nearest().regridder(cube, ywind_cube)(cube)

ywind2 = ywind_ds.isel(time=slice(2)).sel(pressure=850.0).rename({"y_wind": "ywind"}).copy()
ywind_data = dict(ywind2.data_vars)

# ywind2['ywind'] = (list(map(lambda x: x.name(), ywind_regrid_cube.dim_coords)), ywind_regrid_cube.data)

da = xr.DataArray.from_iris(ywind_regrid_cube).load()
da.attrs = (ywind_ds.isel(time=slice(2)).sel(pressure=850.0).rename({"y_wind": "ywind"}).data_vars["ywind"].attrs)
da.encoding = (ywind_ds.isel(time=slice(2)).sel(pressure=850.0).rename({"y_wind": "ywind"}).data_vars["ywind"].encoding)
da

ds = ywind2
print(ds['ywind'].values)
ds['ywind'].values = ywind_regrid_cube.data
# ds = ds.assign({'ywind': da})
# ds['ywind'] = da
# # ds = xr.Dataset(coords = ywind2.coords, data_vars = ywind_data).assign_attrs(ywind_ds.attrs)
# # ds = ywind2
# # ds.ywind.values
reopen_as_cube(ds)
# ds
ds["ywind"].values

[[[-1.62966309e+01 -1.63251953e+01 -1.63864746e+01 ...  2.90966797e+00
    3.09472656e+00  3.28222656e+00]
  [-1.63381348e+01 -1.63676758e+01 -1.64296875e+01 ...  2.89843750e+00
    3.08837891e+00  3.28930664e+00]
  [-1.63823242e+01 -1.64121094e+01 -1.64738770e+01 ...  2.88305664e+00
    3.07250977e+00  3.27709961e+00]
  ...
  [-1.15048828e+01 -1.14208984e+01 -1.12385254e+01 ...  0.00000000e+00
    0.00000000e+00  0.00000000e+00]
  [-1.14394531e+01 -1.13486328e+01 -1.11586914e+01 ...  0.00000000e+00
    0.00000000e+00  0.00000000e+00]
  [-1.13676758e+01 -1.12734375e+01 -1.10815430e+01 ...  0.00000000e+00
    0.00000000e+00  0.00000000e+00]]

 [[-4.58813477e+00 -4.52661133e+00 -4.41796875e+00 ... -4.31640625e+00
   -4.24316406e+00 -4.16723633e+00]
  [-4.52416992e+00 -4.45947266e+00 -4.34252930e+00 ... -4.32324219e+00
   -4.24169922e+00 -4.15380859e+00]
  [-4.43847656e+00 -4.37207031e+00 -4.24804688e+00 ... -4.32861328e+00
   -4.24047852e+00 -4.15014648e+00]
  ...
  [-8.58740234e+00 -8.5

array([[[-16.652172  , -16.652172  , -16.652172  , ...,   2.2106857 ,
           2.2106857 ,   2.2106857 ],
        [-16.652172  , -16.652172  , -16.652172  , ...,   2.2106857 ,
           2.2106857 ,   2.2106857 ],
        [-16.652172  , -16.652172  , -16.652172  , ...,   2.2106857 ,
           2.2106857 ,   2.2106857 ],
        ...,
        [-11.616539  , -11.616539  , -11.616539  , ...,   0.20346451,
           0.20346451,   0.20346451],
        [-11.616539  , -11.616539  , -11.616539  , ...,   0.20346451,
           0.20346451,   0.20346451],
        [-11.616539  , -11.616539  , -11.616539  , ...,   0.20346451,
           0.20346451,   0.20346451]],

       [[ -3.8556252 ,  -3.8556252 ,  -3.8556252 , ...,  -4.8850555 ,
          -4.8850555 ,  -4.8850555 ],
        [ -3.8556252 ,  -3.8556252 ,  -3.8556252 , ...,  -4.8850555 ,
          -4.8850555 ,  -4.8850555 ],
        [ -3.8556252 ,  -3.8556252 ,  -3.8556252 , ...,  -4.8850555 ,
          -4.8850555 ,  -4.8850555 ],
        ...,


In [226]:
da

<xarray.DataArray 'ywind' (time: 2, grid_latitude: 655, grid_longitude: 532)>
array([[[-16.652172  , -16.652172  , -16.652172  , ...,   2.2106857 ,
           2.2106857 ,   2.2106857 ],
        [-16.652172  , -16.652172  , -16.652172  , ...,   2.2106857 ,
           2.2106857 ,   2.2106857 ],
        [-16.652172  , -16.652172  , -16.652172  , ...,   2.2106857 ,
           2.2106857 ,   2.2106857 ],
        ...,
        [-11.616539  , -11.616539  , -11.616539  , ...,   0.20346451,
           0.20346451,   0.20346451],
        [-11.616539  , -11.616539  , -11.616539  , ...,   0.20346451,
           0.20346451,   0.20346451],
        [-11.616539  , -11.616539  , -11.616539  , ...,   0.20346451,
           0.20346451,   0.20346451]],

       [[ -3.8556252 ,  -3.8556252 ,  -3.8556252 , ...,  -4.8850555 ,
          -4.8850555 ,  -4.8850555 ],
        [ -3.8556252 ,  -3.8556252 ,  -3.8556252 , ...,  -4.8850555 ,
          -4.8850555 ,  -4.8850555 ],
        [ -3.8556252 ,  -3.8556252 ,  -3.8556252 , ...,  -4.8850555 ,
          -4.8850555 ,  -4.8850555 ],
        ...,
        [ -9.456966  ,  -9.456966  ,  -9.456966  , ...,   0.33029175,
           0.33029175,   0.33029175],
        [ -9.456966  ,  -9.456966  ,  -9.456966  , ...,   0.33029175,
           0.33029175,   0.33029175],
        [ -9.456966  ,  -9.456966  ,  -9.456966  , ...,   0.33029175,
           0.33029175,   0.33029175]]], dtype=float32)
Coordinates:
  * time                     (time) object 1980-12-01 12:00:00 1980-12-02 12:...
  * grid_latitude            (grid_latitude) float32 -5.565 -5.529 ... 8.945
  * grid_longitude           (grid_longitude) float32 353.1 353.1 ... 365.2
    forecast_reference_time  object 1979-09-01 00:00:00
    realization              int64 1100000
    forecast_period          (time) timedelta64[ns] 450 days 12:00:00 451 day...
Attributes:
    standard_name:    y_wind
    units:            m s-1
    um_stash_source:  m01s30i202
    cell_methods:     time: mean (interval: 1 hour)
    grid_mapping:     rotated_latitude_longitude

In [182]:
ywind_ds.isel(time=slice(2)).sel(pressure=850.0).rename({"y_wind": "ywind"}).grid_latitude_bnds.encoding

{'zlib': False,
 'shuffle': False,
 'complevel': 0,
 'fletcher32': False,
 'contiguous': True,
 'chunksizes': None,
 'source': '/home/henry/cdt/summer/data/moose/uk/2.2km/rcp85/01/ywind/day/ywind_rcp85_land-cpm_uk_2.2km_01_day_19801201-19811130.nc',
 'original_shape': (655, 2),
 'dtype': dtype('float32')}

In [183]:
ywind2.grid_latitude_bnds.encoding

{'zlib': False,
 'shuffle': False,
 'complevel': 0,
 'fletcher32': False,
 'contiguous': True,
 'chunksizes': None,
 'source': '/home/henry/cdt/summer/data/moose/uk/2.2km/rcp85/01/ywind/day/ywind_rcp85_land-cpm_uk_2.2km_01_day_19801201-19811130.nc',
 'original_shape': (655, 2),
 'dtype': dtype('float32')}

In [178]:
ywind_ds['y_wind'].encoding

{'zlib': False,
 'shuffle': False,
 'complevel': 0,
 'fletcher32': False,
 'contiguous': True,
 'chunksizes': None,
 'source': '/home/henry/cdt/summer/data/moose/uk/2.2km/rcp85/01/ywind/day/ywind_rcp85_land-cpm_uk_2.2km_01_day_19801201-19811130.nc',
 'original_shape': (360, 17, 655, 532),
 'dtype': dtype('float32'),
 'coordinates': 'forecast_period forecast_reference_time realization'}

In [170]:
ywind_regrid_cube

<iris 'Cube' of y_wind / (m s-1) (time: 2; grid_latitude: 655; grid_longitude: 532)>

In [162]:
ds

<xarray.Dataset>
Dimensions:                     (time: 2, grid_latitude: 655,
                                 grid_longitude: 532, bnds: 2)
Coordinates:
  * time                        (time) object 1980-12-01 12:00:00 1980-12-02 ...
    pressure                    float32 850.0
  * grid_latitude               (grid_latitude) float32 -5.565 -5.529 ... 8.945
  * grid_longitude              (grid_longitude) float32 353.1 353.1 ... 365.2
    forecast_period             (time) timedelta64[ns] 450 days 12:00:00 451 ...
    forecast_reference_time     object 1979-09-01 00:00:00
    realization                 int32 1100000
Dimensions without coordinates: bnds
Data variables:
    ywind                       (time, grid_latitude, grid_longitude) float32 ...
    rotated_latitude_longitude  int32 -2147483647
    time_bnds                   (time, bnds) object 1980-12-01 00:00:00 ... 1...
    grid_latitude_bnds          (grid_latitude, bnds) float32 -5.583 ... 1.07...
    grid_longitude_bnds         (grid_longitude, bnds) float32 353.1 ... 365.2
    forecast_period_bnds        (time, bnds) float64 1.08e+04 ... 1.085e+04

In [154]:
ywind_ds.isel(time=slice(2)).sel(pressure=850.0).rename({"y_wind": "ywind"}).data_vars["ywind"].attrs

{'standard_name': 'y_wind',
 'units': 'm s-1',
 'um_stash_source': 'm01s30i202',
 'cell_methods': 'time: mean (interval: 1 hour)',
 'grid_mapping': 'rotated_latitude_longitude'}

In [158]:
da = xr.DataArray.from_iris(ywind_regrid_cube)
da.attrs = (ywind_ds.isel(time=slice(2)).sel(pressure=850.0).rename({"y_wind": "ywind"}).data_vars["ywind"].attrs)
da

<xarray.DataArray 'ywind' (time: 2, grid_latitude: 655, grid_longitude: 532)>
dask.array<filled, shape=(2, 655, 532), dtype=float32, chunksize=(2, 655, 532), chunktype=numpy.ndarray>
Coordinates:
  * time                     (time) object 1980-12-01 12:00:00 1980-12-02 12:...
  * grid_latitude            (grid_latitude) float32 -5.565 -5.529 ... 8.945
  * grid_longitude           (grid_longitude) float32 353.1 353.1 ... 365.2
    forecast_reference_time  object ...
    realization              int64 ...
    forecast_period          (time) timedelta64[ns] ...
Attributes:
    standard_name:  y_wind
    units:          m s-1
    source:         Data from Met Office Unified Model
    um_version:     10.6
    Conventions:    CF-1.7
    STASH:          m01s30i202
    cell_methods:   time: mean (interval: 1 hour)

In [91]:
from ml_downscaling_emulator.preprocessing.select_domain import SelectDomain

In [124]:
# SelectDomain(subdomain="london").run(ds)

centre_ds = ds.sel(grid_longitude=360.0+SelectDomain.LONDON_RP_LONG_LAT[0], grid_latitude=SelectDomain.LONDON_RP_LONG_LAT[1], method="nearest")
centre_long_idx = np.where(ds.grid_longitude.values == centre_ds.grid_longitude.values)[0][0]
centre_lat_idx = np.where(ds.grid_latitude.values == centre_ds.grid_latitude.values)[0][0]
ds = ds.sel(grid_longitude=slice(ds.grid_longitude[centre_long_idx-31].values, ds.grid_longitude[centre_long_idx+32].values), grid_latitude=slice(ds.grid_latitude[centre_lat_idx-31].values, ds.grid_latitude[centre_lat_idx+32].values))

ds

<xarray.Dataset>
Dimensions:                     (time: 2, grid_latitude: 64,
                                 grid_longitude: 64, bnds: 2)
Coordinates:
  * time                        (time) object 1980-12-01 12:00:00 1980-12-02 ...
  * grid_latitude               (grid_latitude) float32 -1.59 -1.57 ... -0.33
  * grid_longitude              (grid_longitude) float32 360.9 360.9 ... 362.1
    forecast_reference_time     object 1979-09-01 00:00:00
    realization                 int64 1100000
    forecast_period             (time) timedelta64[ns] 450 days 12:00:00 451 ...
    pressure                    float32 850.0
Dimensions without coordinates: bnds
Data variables:
    ywind                       (time, grid_latitude, grid_longitude) float32 dask.array<chunksize=(2, 64, 64), meta=np.ndarray>
    rotated_latitude_longitude  int32 -2147483647
    time_bnds                   (time, bnds) object 1980-12-01 00:00:00 ... 1...
    grid_latitude_bnds          (grid_latitude, bnds) float32 -1.6 ... -0.32
    grid_longitude_bnds         (grid_longitude, bnds) float32 360.9 ... 362.1
    forecast_period_bnds        (time, bnds) float64 1.08e+04 ... 1.085e+04

In [200]:
iris.load('/home/henry/cdt/summer/code/ml-downscaling-emulation/src/ml_downscaling_emulator/utils/moose_uk_pr_guide-grid.nc')

/home/henry/anaconda3/envs/downscaling/lib/python3.9/site-packages/iris/fileformats/_nc_load_rules/helpers.py:645: UserWarning: Ignoring netCDF variable 'pr' invalid units 'kgm-2s-1'
  warnings.warn(msg)


[<iris 'Cube' of rotated_latitude_longitude / (unknown) (scalar cube)>,
<iris 'Cube' of pr / (unknown) (time: 2; grid_latitude: 654; grid_longitude: 532)>]

In [203]:
xr.open_dataset('/home/henry/cdt/summer/code/ml-downscaling-emulation/src/ml_downscaling_emulator/utils/moose_uk_pr_guide-grid.nc').pr.attrs

{'units': 'kgm-2s-1'}

In [204]:
da.attrs

{'standard_name': 'y_wind',
 'units': 'm s-1',
 'um_stash_source': 'm01s30i202',
 'cell_methods': 'time: mean (interval: 1 hour)',
 'grid_mapping': 'rotated_latitude_longitude'}

In [217]:
ds = xr.open_dataset(lsrain_meta.filepath(year)).isel(time=slice(2))
ds = ds.assign({'pr': lambda x: x['stratiform_rainfall_flux']})
ds = ds.drop_vars(['stratiform_rainfall_flux'])
ds['pr'] = ds['pr'].assign_attrs({"units": "kg m-2 s-1"})
ds
reopen_as_cube(ds)

<iris 'Cube' of stratiform_rainfall_flux / (kg m-2 s-1) (time: 2; grid_latitude: 654; grid_longitude: 532)>

In [215]:
ds['pr'].attrs

{'standard_name': 'stratiform_rainfall_flux',
 'units': 'kg m-2 s-1',
 'um_stash_source': 'm01s04i203',
 'cell_methods': 'time: mean (interval: 1 hour)',
 'grid_mapping': 'rotated_latitude_longitude'}